In [8]:
# 必要なライブラリをインポート 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
# s3 からデータを読み込む

import boto3

# s3のパスを指定
s3_bucket_path = 's3://kikagaku-handson-10'

# 使用するファイルパスを指定
train_path = 'iris_data.csv'

# 読み込み用のS3パスを結合
train_path = f"{s3_bucket_path}/{train_path}"

# pandasでCSVを読み込み
df_train = pd.read_csv(train_path)
df_train.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


データの詳細
| 特徴   | 内容                                                                                                  |
| ---- | --------------------------------------------------------------------------------------------------- |
| データ数 | 150件                                                                                                |
| クラス数 | 3種類（setosa, versicolor, virginica）                                                                  |
| 特徴量  | 4つの実数値：<br>sepal length（がく片の長さ）<br>sepal width（がく片の幅）<br>petal length（花びらの長さ）<br>petal width（花びらの幅） |
| 目的   | **3クラス分類**（線形・非線形の分類手法の比較に最適）                                                                       |



setosa、versicolor、virginic という３種類のあやめ（Iris）のデータがあります。<br>
これを、がく片（sepal）や花弁（petal）の長さ・幅といった情報（これを特徴量と呼びます）から分類をしたい、という機械学習モデルを作成します。

In [27]:
# 入力変数と出力変数の切り分け
x = df_train.drop('species', axis=1)
t = df_train['species']

# 2 つに分割
from sklearn.model_selection import train_test_split
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.2, random_state=0)

# モデルの定義
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

# モデルの学習
model.fit(x_train, t_train)

# モデルの検証
print(model.score(x_train, t_train))
print(model.score(x_test, t_test))

0.9666666666666667
1.0


## S3への保存
ここまでで、あやめの分類を行いました。
ここからは、分類結果をcsv形式で保存し、そのデータを S3 へ格納する様子を見てみましょう。

In [28]:
# 予測値を取得
y_pred = model.predict(x_test)

# t_testのインデックスを保持したままDataFrameを作成（正解ラベルと予測ラベルを並べる）
df_result = pd.DataFrame({
    'True Label': t_test,
    'Predicted Label': y_pred
})

# ノートブックと同じディレクトリにファイルを保存
df_result.to_csv('classification_result.csv', index=False)

In [29]:
# S3 へアップロードを行う。

import boto3

s3 = boto3.client('s3')
bucket_name = 'kikagaku-handson-10'
file_name = 'classification_result.csv'
 
s3.upload_file(file_name, bucket_name, file_name)

# s3.upload_file()の引数３つについては、以下の意味を指しています。
# 手元の "classification_result.csv" ファイルを、
# S3の "kikagaku-handson-10" バケットに、
# 同じ名前 "classification_result.csv" として保存する。